# Proyecto Redes Neuronales: Perceptrón Multicapa

**Curso:** Redes Neuronales 2025-II  
**Objetivo:** Entrenar y evaluar un modelo de Perceptrón Multicapa (MLP) para clasificación de texto según el enunciado del proyecto.  
**Autor:** Herney Eduardo Quintero Trochez  
**Fecha:** 2025  
**Universidad:** Universidad Del Valle  

## Componentes implementados:
1. Configuración de Parámetros Globales
2. Carga y Preprocesamiento de Datos
3. Tokenización y Creación del Vocabulario
4. Construcción del Modelo MLP
5. Entrenamiento con Early Stopping
6. Evaluación del Modelo
7. Guardado de Resultados y Modelos
8. Visualización de Resultados
9. Historial de Experimentos

## 0. Configuración de Parámetros Globales

Esta sección permite modificar fácilmente todos los parámetros del modelo para experimentar.

In [ ]:
# ===== CONFIGURACIÓN DE PARÁMETROS GLOBALES =====
# Esta sección centraliza todos los parámetros del modelo para facilitar experimentación

# Configuración general del experimento
EXPERIMENT_NAME = "MultiLayer_Perceptron"
MODEL_TYPE = "MLP"  # Perceptrón Multi-Capa  
RANDOM_SEED = 42

# Directorios de trabajo
DATA_DIR = "data"
MODEL_DIR = "models"
OUTPUT_DIR = "output"

# Configuración del dataset - MEJORADO CON TÍTULO
TEXT_COLUMN = "review_body"  # Columna con el texto del cuerpo de la reseña
TITLE_COLUMN = "review_title"  # Columna con el título de la reseña
TARGET_COLUMN = "stars"  # Columna con las etiquetas (1-5 estrellas)
LANGUAGE_COLUMN = "language"  # Columna con el idioma
FILTER_LANGUAGE = "en"  # Filtrar por idioma específico. Opciones: None, "en", "es", "de", "fr", "ja"
USE_TITLE_AND_BODY = True  # NUEVO: Usar tanto título como cuerpo para mejor precisión
MAX_WORDS = 80000  # Tamaño máximo del vocabulario
MAX_LENGTH = 300  # Longitud máxima de las secuencias (aumentada por combinar título+cuerpo)
OOV_TOKEN = "<OOV>"  # Token para palabras fuera del vocabulario

# Parámetros de arquitectura del modelo - MLP
EMBEDDING_DIM = 300  # Dimensión del embedding de palabras
HIDDEN_LAYERS = [256, 128, 64]  # Lista con el número de neuronas en cada capa oculta
ACTIVATION = "relu"  # Función de activación para capas ocultas
OUTPUT_ACTIVATION = "softmax"  # Función de activación para la capa de salida
DROPOUT_RATE = 0.3  # Tasa de dropout para regularización

# Parámetros de entrenamiento
EPOCHS = 50  # Número máximo de épocas de entrenamiento
BATCH_SIZE = 512  # Tamaño del batch
LEARNING_RATE = 0.001  # Tasa de aprendizaje
PATIENCE = 10  # Paciencia para early stopping
OPTIMIZER = "adam"  # Optimizador a usar
LOSS_FUNCTION = "categorical_crossentropy"  # Función de pérdida
METRICS = ["accuracy"]  # Métricas a monitorear

print(f"=== Configuración del Experimento: {EXPERIMENT_NAME} ===")
print(f"Modelo: {MODEL_TYPE}")
print(f"Filtro de idioma: {FILTER_LANGUAGE if FILTER_LANGUAGE else 'Multiidioma'}")
print(f"Usar título + cuerpo: {USE_TITLE_AND_BODY}")
print(f"Longitud máxima: {MAX_LENGTH} tokens")
print(f"Arquitectura oculta: {HIDDEN_LAYERS}")
print(f"Dimensión embedding: {EMBEDDING_DIM}")
print(f"Dropout: {DROPOUT_RATE}")
print(f"Épocas máximas: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Tasa de aprendizaje: {LEARNING_RATE}")
print("="*60)

## 1. Importación de Librerías y Funciones Helper

In [ ]:
# Librerías estándar
import os
import numpy as np
from datetime import datetime

# Librerías de machine learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

# Importar funciones helper
from helper import (
    DataLoader, ModelTrainer, ResultsManager, Visualizer,
    evaluate_model, get_gpu_info, setup_experiment_environment
)

# Configurar ambiente del experimento
gpu_info = setup_experiment_environment(RANDOM_SEED)
print(f"Ambiente configurado. GPU disponible: {gpu_info['gpu_available']}")

## 2. Carga y Preprocesamiento de Datos

In [ ]:
# Inicializar el cargador de datos
data_loader = DataLoader(data_dir=DATA_DIR)

# Cargar los datasets
print("Cargando datasets...")
train_df, val_df, test_df = data_loader.load_all_data()

print(f"\nDatos originales cargados:")
print(f"Entrenamiento: {len(train_df)} muestras")
print(f"Validación: {len(val_df)} muestras") 
print(f"Prueba: {len(test_df)} muestras")

# Verificar que las columnas necesarias existen
required_columns = [TEXT_COLUMN, TARGET_COLUMN, LANGUAGE_COLUMN]
if USE_TITLE_AND_BODY:
    required_columns.append(TITLE_COLUMN)

missing_columns = [col for col in required_columns if col not in train_df.columns]
if missing_columns:
    print(f"Columnas faltantes: {missing_columns}")
    print(f"Columnas disponibles: {list(train_df.columns)}")
else:
    print(f"Todas las columnas requeridas están disponibles")
    if USE_TITLE_AND_BODY:
        print(f"Modo combinado: {TITLE_COLUMN} + {TEXT_COLUMN}")

# Analizar distribución de idiomas
print(f"\nAnálisis de idiomas en el dataset:")
if LANGUAGE_COLUMN in train_df.columns:
    lang_dist_train = train_df[LANGUAGE_COLUMN].value_counts()
    print(f"Distribución de idiomas (entrenamiento):")
    for lang, count in lang_dist_train.items():
        percentage = (count / len(train_df)) * 100
        print(f"  {lang}: {count:,} muestras ({percentage:.1f}%)")
    
    # Aplicar filtro por idioma si está especificado
    if FILTER_LANGUAGE is not None:
        if FILTER_LANGUAGE in lang_dist_train.index:
            print(f"\nFiltrando por idioma: {FILTER_LANGUAGE}")
            
            # Filtrar datasets por idioma
            train_df = train_df[train_df[LANGUAGE_COLUMN] == FILTER_LANGUAGE].copy()
            val_df = val_df[val_df[LANGUAGE_COLUMN] == FILTER_LANGUAGE].copy()
            test_df = test_df[test_df[LANGUAGE_COLUMN] == FILTER_LANGUAGE].copy()
            
            print(f"\nDatos después del filtrado por idioma '{FILTER_LANGUAGE}':")
            print(f"Entrenamiento: {len(train_df)} muestras")
            print(f"Validación: {len(val_df)} muestras")
            print(f"Prueba: {len(test_df)} muestras")
        else:
            print(f"\nAdvertencia: Idioma '{FILTER_LANGUAGE}' no encontrado en el dataset.")
            print(f"Idiomas disponibles: {list(lang_dist_train.index)}")
            print("Usando todos los idiomas...")
    else:
        print(f"\nUsando todos los idiomas disponibles")
else:
    print(f"Columna '{LANGUAGE_COLUMN}' no encontrada. Usando todos los datos sin filtrar.")

# Mostrar distribución de clases en el conjunto final
print(f"\nDistribución de clases (conjunto final):")
class_distribution = train_df[TARGET_COLUMN].value_counts().sort_index()
for stars, count in class_distribution.items():
    percentage = (count / len(train_df)) * 100
    print(f"  {stars} estrella(s): {count:,} muestras ({percentage:.1f}%)")

# Mostrar ejemplos de datos con título y cuerpo
print(f"\nEjemplos de datos del conjunto final:")
for i in range(min(3, len(train_df))):
    lang = train_df[LANGUAGE_COLUMN].iloc[i] if LANGUAGE_COLUMN in train_df.columns else "N/A"
    title = train_df[TITLE_COLUMN].iloc[i] if USE_TITLE_AND_BODY and TITLE_COLUMN in train_df.columns else "N/A"
    text = train_df[TEXT_COLUMN].iloc[i][:80]  # Menos texto para mostrar título también
    stars = train_df[TARGET_COLUMN].iloc[i]
    
    print(f"{i+1}. [{lang}] {stars} estrella(s)")
    if USE_TITLE_AND_BODY and title != "N/A":
        print(f"   Título: {title[:60]}{'...' if len(str(title)) > 60 else ''}")
    print(f"   Cuerpo: {text}{'...' if len(str(train_df[TEXT_COLUMN].iloc[i])) > 80 else ''}")
    print()

### Configuraciones Rápidas por Idioma

Para cambiar el idioma del experimento, modifica la variable `FILTER_LANGUAGE` en la primera celda:

```python
# Opciones disponibles:
FILTER_LANGUAGE = None     # Todos los idiomas (por defecto)
FILTER_LANGUAGE = "en"     # Solo inglés 
FILTER_LANGUAGE = "es"     # Solo español
FILTER_LANGUAGE = "de"     # Solo alemán
FILTER_LANGUAGE = "fr"     # Solo francés
FILTER_LANGUAGE = "ja"     # Solo japonés
```

**Recomendaciones:**
- **`"en"` (Inglés)**: Mayor cantidad de datos, vocabulario más rico
- **`"es"` (Español)**: Bueno para comparar rendimiento en español
- **`None` (Todos)**: Para experimentos multiidioma
- **Otros idiomas**: Para análisis específicos por idioma

In [ ]:
# CONFIGURACIÓN RÁPIDA - Descomenta una línea para cambiar idioma
# FILTER_LANGUAGE = None      # Por defecto: todos los idiomas
# FILTER_LANGUAGE = "en"      # Solo inglés (recomendado para mejor rendimiento)
# FILTER_LANGUAGE = "es"      # Solo español 
# FILTER_LANGUAGE = "de"      # Solo alemán
# FILTER_LANGUAGE = "fr"      # Solo francés
# FILTER_LANGUAGE = "ja"      # Solo japonés

# Si cambias el idioma aquí, ejecuta esta celda y vuelve a ejecutar desde la carga de datos
print(f"Configuración actual: {FILTER_LANGUAGE if FILTER_LANGUAGE else 'Todos los idiomas'}")
print("Tip: Para inglés únicamente (mejor rendimiento), descomenta: FILTER_LANGUAGE = 'en'")

## 3. Preprocesamiento de Texto

In [ ]:
# Preprocesar los datos de texto
print("Preprocesando datos de texto...")
print(f"Modo: {'Título + Cuerpo' if USE_TITLE_AND_BODY else 'Solo Cuerpo'}")

processed_data = data_loader.preprocess_text_data_embedding(
    train_df=train_df,
    val_df=val_df, 
    test_df=test_df,
    text_column=TEXT_COLUMN,
    title_column=TITLE_COLUMN if USE_TITLE_AND_BODY else None,
    target_column=TARGET_COLUMN,
    max_words=MAX_WORDS,
    max_length=MAX_LENGTH,
    use_title_and_body=USE_TITLE_AND_BODY
)

# Extraer datos preprocesados
X_train, y_train = processed_data['X_train'], processed_data['y_train']
X_val, y_val = processed_data['X_val'], processed_data['y_val']
X_test, y_test = processed_data['X_test'], processed_data['y_test']
num_classes = processed_data['num_classes']
vocab_size = processed_data['vocab_size']

print(f"\nDatos preprocesados:")
print(f"Tamaño del vocabulario: {vocab_size}")
print(f"Número de clases: {num_classes}")
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Texto combinado: {'Sí (título + cuerpo)' if USE_TITLE_AND_BODY else 'No (solo cuerpo)'}")

# Obtener nombres de clases para evaluación
class_names = [str(i) for i in data_loader.label_encoder.classes_]

### Mejora Implementada: Título + Cuerpo

**Ventajas de combinar título y cuerpo:**
- **Más contexto:** El título often contiene información clave sobre el sentiment
- **Mejor precisión:** Más información textual para el modelo
- **Vocabulario enriquecido:** Palabras clave del título complementan el cuerpo

In [ ]:
# Veamos algunos ejemplos de cómo se ve el texto combinado
print("EJEMPLOS DE TEXTO COMBINADO (Título + Cuerpo):")
print("=" * 60)

# Seleccionar algunos ejemplos del conjunto de entrenamiento
for i in range(3):
    title = train_df.iloc[i]['review_title']
    body = train_df.iloc[i]['review_body']
    combined = f"{title} {body}"
    stars = train_df.iloc[i]['stars']
    
    print(f"\nEjemplo {i+1} - {stars} estrella(s):")
    print(f"Título: {title}")
    print(f"Cuerpo: {body[:100]}...")
    print(f"Combinado: {combined[:150]}...")
    print("-" * 40)

print(f"\nBeneficios de la combinación:")
print(f"   Mayor contexto semántico")
print(f"   Información clave del título se preserva")  
print(f"   Vocabulario más rico para el modelo")
print(f"   Longitud máxima aumentada a {MAX_LENGTH} tokens")

### Función para Comparar Configuraciones

Esta celda permite cambiar fácilmente entre usar solo el cuerpo o título+cuerpo:

In [ ]:
# CONFIGURACIÓN RÁPIDA - Cambia aquí para experimentar
print("CONFIGURACIONES DISPONIBLES:")
print("=" * 50)
print("1. Título + Cuerpo (ACTUAL)")
print("   - USE_TITLE_AND_BODY = True")
print("   - MAX_LENGTH = 300")
print("   - Mejor precisión esperada")
print()
print("2. Solo Cuerpo (TRADICIONAL)") 
print("   - USE_TITLE_AND_BODY = False")
print("   - MAX_LENGTH = 250")
print("   - Más rápido de procesar")
print()
print(f"Configuración actual: {'Título + Cuerpo' if USE_TITLE_AND_BODY else 'Solo Cuerpo'}")
print(f"Longitud máxima: {MAX_LENGTH} tokens")
print(f"Vocabulario: {vocab_size:,} palabras")
print()
print("Para cambiar configuración:")
print("   1. Modifica USE_TITLE_AND_BODY en la primera celda")
print("   2. Ajusta MAX_LENGTH según necesites")
print("   3. Re-ejecuta desde la carga de datos")

### Comparación: Embedding vs BoW/TF-IDF

**Nuevo:** Ahora puedes elegir entre dos enfoques de preprocesamiento:
- **Embedding (usado arriba):** Secuencias numéricas para capas de embedding
- **BoW/TF-IDF:** Matrices dispersas optimizadas en memoria

In [ ]:
# DEMOSTRACIÓN: Comparación de enfoques de preprocesamiento
print("COMPARACIÓN DE ENFOQUES DE PREPROCESAMIENTO")
print("=" * 60)

# Crear un data loader separado para la comparación
comparison_loader = DataLoader(data_dir=DATA_DIR)

# Usar una muestra pequeña para demostración rápida
sample_size = min(1000, len(train_df))
train_sample = train_df.sample(sample_size).reset_index(drop=True)
val_sample = val_df.sample(min(200, len(val_df))).reset_index(drop=True)
test_sample = test_df.sample(min(200, len(test_df))).reset_index(drop=True)

print(f"Usando muestra de {sample_size} registros de entrenamiento para comparación rápida...\n")

# 1. Enfoque Embedding (ya usado arriba)
print("1. ENFOQUE EMBEDDING (actual):")
print("-" * 30)
embedding_data = comparison_loader.preprocess_text_data_embedding(
    train_sample, val_sample, test_sample,
    text_column=TEXT_COLUMN,
    title_column=TITLE_COLUMN if USE_TITLE_AND_BODY else None,
    target_column=TARGET_COLUMN,
    max_words=MAX_WORDS,
    max_length=MAX_LENGTH,
    use_title_and_body=USE_TITLE_AND_BODY
)

embedding_memory = embedding_data['X_train'].nbytes / (1024 * 1024)
print(f"   Memoria: {embedding_memory:.1f} MB")
print(f"   Forma: {embedding_data['X_train'].shape}")
print(f"   Tipo: Matriz densa (numpy array)")

print("\n2. ENFOQUE BOW/TF-IDF (optimizado):")
print("-" * 35)
bow_data = comparison_loader.preprocess_text_data_bow(
    train_sample, val_sample, test_sample,
    text_column=TEXT_COLUMN,
    title_column=TITLE_COLUMN if USE_TITLE_AND_BODY else None,
    target_column=TARGET_COLUMN,
    max_features=5000,  # Optimizado según especificaciones
    min_df=3,
    max_df=0.85,
    use_tfidf=True,
    use_title_and_body=USE_TITLE_AND_BODY
)

print(f"   Memoria: {bow_data['memory_mb']:.1f} MB")
print(f"   Forma: {bow_data['X_train'].shape}")
print(f"   Tipo: Matriz dispersa (scipy sparse)")
print(f"   Sparsity: {bow_data['sparsity']:.1%}")

# Comparación final
print("\n" + "=" * 60)
print("RESUMEN DE COMPARACIÓN:")
print("=" * 60)

if embedding_memory > 0:
    memory_savings = embedding_memory - bow_data['memory_mb']
    savings_pct = (memory_savings / embedding_memory) * 100
    print(f"💾 Ahorro de memoria: {memory_savings:.1f} MB ({savings_pct:.1f}%)")
else:
    print(f"💾 BoW usa: {bow_data['memory_mb']:.1f} MB (vs embedding)")

print(f"📊 Características: {bow_data['vocab_size']} (BoW) vs {embedding_data['vocab_size']} (Embedding)")
print(f"🎯 Sparsity: {bow_data['sparsity']:.1%} (BoW es {bow_data['sparsity']:.1%} vacío)")
print(f"⚡ Optimizaciones BoW: vocabulario limitado, matrices dispersas, filtrado de términos")

print("\n💡 Recomendación: Usar BoW para datasets grandes (>100K muestras) por eficiencia de memoria")

## 4. Construcción del Modelo MLP

In [ ]:
def create_mlp_model(vocab_size, embedding_dim, max_length, hidden_layers, 
                     num_classes, dropout_rate, activation, output_activation):
    """
    Crear un modelo de Perceptrón Multicapa para clasificación de texto.
    
    Args:
        vocab_size: Tamaño del vocabulario
        embedding_dim: Dimensión del embedding
        max_length: Longitud máxima de secuencia
        hidden_layers: Lista con el número de neuronas en cada capa oculta
        num_classes: Número de clases de salida
        dropout_rate: Tasa de dropout
        activation: Función de activación para capas ocultas
        output_activation: Función de activación para la capa de salida
    
    Returns:
        tf.keras.Model: Modelo compilado
    """
    model = Sequential()
    
    # Capa de embedding (sin input_length que está deprecado)
    model.add(Embedding(input_dim=vocab_size, 
                       output_dim=embedding_dim,
                       name="embedding_layer"))
    
    # Pooling global para reducir dimensionalidad
    model.add(GlobalAveragePooling1D(name="global_avg_pooling"))
    
    # Capas ocultas del MLP
    for i, units in enumerate(hidden_layers):
        model.add(Dense(units=units, 
                       activation=activation, 
                       name=f"dense_layer_{i+1}"))
        model.add(Dropout(rate=dropout_rate, 
                         name=f"dropout_{i+1}"))
    
    # Capa de salida
    model.add(Dense(units=num_classes, 
                   activation=output_activation, 
                   name="output_layer"))
    
    return model

# Crear el modelo
print("Creando modelo MLP...")
model = create_mlp_model(
    vocab_size=vocab_size,
    embedding_dim=EMBEDDING_DIM,
    max_length=MAX_LENGTH,
    hidden_layers=HIDDEN_LAYERS,
    num_classes=num_classes,
    dropout_rate=DROPOUT_RATE,
    activation=ACTIVATION,
    output_activation=OUTPUT_ACTIVATION
)

# Compilar el modelo
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(
    optimizer=optimizer,
    loss=LOSS_FUNCTION,
    metrics=METRICS
)

# Construir el modelo con la forma de entrada específica
model.build(input_shape=(None, MAX_LENGTH))

# Mostrar arquitectura del modelo
print("\nArquitectura del modelo:")
model.summary()

# Contar parámetros
total_params = model.count_params()
print(f"\nTotal de parámetros: {total_params:,}")

## 5. Entrenamiento del Modelo

In [ ]:
# Inicializar el entrenador de modelos
model_trainer = ModelTrainer(model_dir=MODEL_DIR)

# Entrenar el modelo
print("Iniciando entrenamiento...")
training_results = model_trainer.train_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    patience=PATIENCE,
    model_name=f"{MODEL_TYPE}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
)

print(f"\nEntrenamiento completado:")
print(f"Épocas entrenadas: {training_results['epochs_trained']}")
print(f"Tiempo de entrenamiento: {training_results['training_time']:.1f} segundos")
print(f"Accuracy final (entrenamiento): {training_results['final_train_accuracy']:.4f}")
print(f"Accuracy final (validación): {training_results['final_val_accuracy']:.4f}")

## 6. Visualización del Entrenamiento

In [ ]:
# Visualizar el historial de entrenamiento
Visualizer.plot_training_history(
    history=training_results['history'],
    model_name=MODEL_TYPE,
    save_path=os.path.join(OUTPUT_DIR, f"{MODEL_TYPE}_training_history.png")
)

## 7. Evaluación del Modelo

In [ ]:
# Evaluar el modelo en el conjunto de prueba
evaluation_results = evaluate_model(
    model=model,
    X_test=X_test,
    y_test=y_test,
    class_names=class_names
)

# Extraer métricas de evaluación
test_accuracy = evaluation_results['test_accuracy']
test_loss = evaluation_results['test_loss']
classification_rep = evaluation_results['classification_report']
y_true = evaluation_results['y_true']
y_pred = evaluation_results['y_pred']

print(f"\nResultados en conjunto de prueba:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Loss: {test_loss:.4f}")
print(f"F1-Score (macro): {classification_rep['macro avg']['f1-score']:.4f}")
print(f"F1-Score (weighted): {classification_rep['weighted avg']['f1-score']:.4f}")

## 8. Matriz de Confusión

In [ ]:
# Visualizar matriz de confusión
Visualizer.plot_confusion_matrix(
    y_true=y_true,
    y_pred=y_pred,
    class_names=class_names,
    model_name=MODEL_TYPE,
    save_path=os.path.join(OUTPUT_DIR, f"{MODEL_TYPE}_confusion_matrix.png")
)

## 9. Guardado de Resultados del Experimento

In [ ]:
# Preparar datos del experimento para guardar
experiment_data = {
    'experiment_name': EXPERIMENT_NAME,
    'configuration': {
        'model_type': MODEL_TYPE,
        'text_column': TEXT_COLUMN,
        'target_column': TARGET_COLUMN,
        'language_filter': FILTER_LANGUAGE,  # ← Nueva información de idioma
        'max_words': MAX_WORDS,
        'max_length': MAX_LENGTH,
        'embedding_dim': EMBEDDING_DIM,
        'hidden_layers': HIDDEN_LAYERS,
        'activation': ACTIVATION,
        'output_activation': OUTPUT_ACTIVATION,
        'dropout_rate': DROPOUT_RATE,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'patience': PATIENCE,
        'optimizer': OPTIMIZER,
        'loss_function': LOSS_FUNCTION,
        'random_seed': RANDOM_SEED,
        'total_parameters': total_params,
        'gpu_used': gpu_info['gpu_available']
    },
    'dataset_info': {
        'train_samples': len(train_df),
        'val_samples': len(val_df),
        'test_samples': len(test_df),
        'num_classes': num_classes,
        'vocab_size': vocab_size,
        'class_distribution': class_distribution.to_dict(),
        'language_used': FILTER_LANGUAGE if FILTER_LANGUAGE else "multilingual"  # ← Nueva información
    },
    'training_results': training_results,
    'evaluation_metrics': {
        'test_accuracy': test_accuracy,
        'test_loss': test_loss,
        'f1_macro': classification_rep['macro avg']['f1-score'],
        'f1_weighted': classification_rep['weighted avg']['f1-score'],
        'precision_macro': classification_rep['macro avg']['precision'],
        'recall_macro': classification_rep['macro avg']['recall'],
        'classification_report': classification_rep
    },
    'gpu_info': gpu_info
}

# Guardar resultados del experimento
results_manager = ResultsManager(output_dir=OUTPUT_DIR)
experiment_id = results_manager.save_experiment_results(experiment_data)

print(f"\nExperimento #{experiment_id} guardado exitosamente.")
print(f"Idioma utilizado: {FILTER_LANGUAGE if FILTER_LANGUAGE else 'Multiidioma'}")
print(f"Modelo guardado en: {training_results['model_path']}")
print(f"Resultados guardados en: {OUTPUT_DIR}/experiment_history.json")

## 10. Resumen del Experimento

In [ ]:
# Mostrar resumen del experimento actual
print(f"RESUMEN DEL EXPERIMENTO #{experiment_id}")
print("=" * 60)
print(f"Modelo: {MODEL_TYPE}")
print(f"Idioma: {FILTER_LANGUAGE if FILTER_LANGUAGE else 'Multiidioma (todos)'}")
print(f"Arquitectura: {HIDDEN_LAYERS}")
print(f"Parámetros totales: {total_params:,}")
print(f"")
print(f"Dataset:")
print(f"  - Entrenamiento: {len(train_df):,} muestras")
print(f"  - Validación: {len(val_df):,} muestras")
print(f"  - Prueba: {len(test_df):,} muestras")
print(f"")
print(f"Entrenamiento:")
print(f"  - Épocas: {training_results['epochs_trained']}/{EPOCHS}")
print(f"  - Tiempo: {training_results['training_time']:.1f}s")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"")
print(f"Resultados:")
print(f"  - Test Accuracy: {test_accuracy:.4f}")
print(f"  - Test Loss: {test_loss:.4f}")
print(f"  - F1-Score (macro): {classification_rep['macro avg']['f1-score']:.4f}")
print(f"  - F1-Score (weighted): {classification_rep['weighted avg']['f1-score']:.4f}")
print(f"")
print(f"Hardware: {'GPU' if gpu_info['gpu_available'] else 'CPU'}")

# Mostrar distribución de clases final
print(f"")
print(f"Distribución de clases utilizadas:")
for stars, count in class_distribution.items():
    percentage = (count / len(train_df)) * 100
    print(f"  {stars} estrella(s): {count:,} ({percentage:.1f}%)")

## 11. Historial de Experimentos

In [ ]:
# Mostrar historial completo de experimentos
results_manager.display_experiment_history()

## 12. Predicciones de Ejemplo

In [ ]:
def predict_sample_texts(model, tokenizer, sample_texts, class_names, max_length):
    """
    Hacer predicciones en textos de ejemplo.
    
    Args:
        model: Modelo entrenado
        tokenizer: Tokenizer usado para entrenar
        sample_texts: Lista de textos de ejemplo
        class_names: Nombres de las clases
        max_length: Longitud máxima de secuencia
    """
    # Procesar textos
    sequences = tokenizer.texts_to_sequences(sample_texts)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequences, maxlen=max_length, padding='post', truncating='post'
    )
    
    # Hacer predicciones
    predictions = model.predict(padded)
    predicted_classes = np.argmax(predictions, axis=1)
    
    print("PREDICCIONES DE EJEMPLO:")
    print("=" * 60)
    
    for i, text in enumerate(sample_texts):
        pred_class = predicted_classes[i]
        confidence = predictions[i][pred_class]
        
        print(f"Texto: {text[:100]}...")
        print(f"Predicción: {class_names[pred_class]} estrellas (confianza: {confidence:.3f})")
        print("-" * 40)

# Ejemplos de textos para probar - MEJORADOS CON TÍTULOS
if USE_TITLE_AND_BODY:
    sample_texts = [
        "Excelente producto Este producto es excelente, muy buena calidad y llegó rápido",
        "Producto terrible Terrible producto, no funciona como se describe", 
        "Producto normal El producto está bien, cumple con lo básico",
        "Producto increíble Increíble calidad, superó mis expectativas completamente",
        "No recomendado No recomiendo este producto, muy mala experiencia"
    ]
    print("Usando formato: [TÍTULO] + [CUERPO] para mejores predicciones")
else:
    sample_texts = [
        "Este producto es excelente, muy buena calidad y llegó rápido",
        "Terrible producto, no funciona como se describe", 
        "El producto está bien, cumple con lo básico",
        "Increíble calidad, superó mis expectativas completamente",
        "No recomiendo este producto, muy mala experiencia"
    ]
    print("Usando formato: solo [CUERPO]")

# Hacer predicciones en ejemplos
predict_sample_texts(
    model=model,
    tokenizer=data_loader.tokenizer,
    sample_texts=sample_texts,
    class_names=class_names,
    max_length=MAX_LENGTH
)

## 13. Análisis de Errores

In [ ]:
# Analizar algunos errores del modelo
def analyze_errors(X_test, y_test, y_pred, test_df, text_column, target_column, n_examples=5):
    """
    Analizar ejemplos donde el modelo se equivocó.
    """
    # Encontrar índices donde el modelo se equivocó
    y_test_labels = np.argmax(y_test, axis=1)
    error_indices = np.where(y_test_labels != y_pred)[0]
    
    if len(error_indices) == 0:
        print("Perfecto. El modelo no cometió errores en el conjunto de prueba.")
        return
    
    print(f"ANÁLISIS DE ERRORES ({len(error_indices)} errores total)")
    print("=" * 60)
    
    # Mostrar algunos ejemplos de errores
    sample_errors = np.random.choice(error_indices, 
                                   min(n_examples, len(error_indices)), 
                                   replace=False)
    
    for i, idx in enumerate(sample_errors):
        true_label = y_test_labels[idx]
        pred_label = y_pred[idx]
        text = test_df.iloc[idx][text_column]
        
        print(f"Error #{i+1}:")
        print(f"Texto: {text[:150]}...")
        print(f"Etiqueta real: {true_label + 1} estrellas")
        print(f"Predicción: {pred_label + 1} estrellas")
        print("-" * 40)

# Realizar análisis de errores
analyze_errors(X_test, y_test, y_pred, test_df, TEXT_COLUMN, TARGET_COLUMN)

## 14. Conclusiones y Próximos Pasos

### Resultados obtenidos:
- **Modelo implementado:** Perceptrón Multicapa (MLP) para clasificación de sentimientos
- **Arquitectura:** Embedding + GlobalAveragePooling + Capas densas + Dropout
- **Dataset:** Reviews de Amazon con clasificación de 1-5 estrellas
- **Métricas principales:** Accuracy, F1-Score, Precision, Recall

### Experimentos sugeridos:
1. **Ajuste de hiperparámetros:**
   - Probar diferentes dimensiones de embedding (50, 100, 200, 300)
   - Experimentar con diferentes arquitecturas de capas ocultas
   - Ajustar tasas de dropout y learning rate

2. **Mejoras del modelo:**
   - Implementar técnicas de regularización adicionales
   - Probar diferentes optimizadores (SGD, RMSprop, AdaGrad)
   - Experimentar con funciones de activación alternativas

3. **Preprocesamiento:**
   - Ajustar el tamaño del vocabulario
   - Experimentar con diferentes longitudes de secuencia
   - Implementar técnicas de limpieza de texto más sofisticadas

### Archivos generados:
- `models/`: Modelos entrenados guardados
- `output/experiment_history.json`: Historial completo de experimentos
- `output/`: Gráficos de entrenamiento y matrices de confusión

### Para la siguiente entrega:
- Reutilizar las funciones del archivo `helper.py`
- Comparar resultados con otras arquitecturas (RNN, CNN)
- Implementar técnicas de ensemble o voting
- Realizar análisis más profundo de los errores del modelo